# Simple binary classification problem utilizing convolutional neural networks 

## Import libraries 

In [4]:
# Import libraries. 
from __future__ import print_function
import os
import sys
#os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu0,floatX=float32,optimizer=fast_compile'
#os.environ['KERAS_BACKEND'] = 'theano'
# """
# os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu3,floatX=float32,optimizer=fast_compile'
# os.environ['KERAS_BACKEND'] = 'theano'

# In case you want to select a graphic card (i the above code i set the 3rd graphic card.) 
# """

from keras.api.models import Sequential
from keras.api.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.api.layers import Convolution2D, MaxPooling2D
from keras.api.optimizers import SGD
from keras.api.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
import numpy as np
import keras 
import keras.api.backend as K
from keras.api.callbacks import LearningRateScheduler
import math
from keras import callbacks
import glob
from PIL import Image
from keras.src.utils import plot_model
import h5py
import time


## Basic functions 

In [5]:
# It is good to know the pid of the running code in case you need to stop  or monitor. 
# print (os.getpid())
import keras.api


file_open = lambda x,y: glob.glob(os.path.join(x,y))

# learning rate schedule. It is helpful when the learning rate can be dynamically set up. We will be using the callback functionality that keras provides. 
def step_decay(epoch):
  initial_lrate = 0.01
  drop = 0.3
  epochs_drop = 30.0
  # This function doesn't actually affect the learning rate too much until a higher number of epochs is reached (around 30)
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  #print("Learning rate:", lrate)
  return lrate

# The following function will be used to give a number of the parameters in our model. Useful when we need to get an estimate of what size of dataset we have to use.  
def size(model): 
  return sum([np.prod(K.get_value(w).shape) for w in model.trainable_weights])

def createmodel(img_rows, img_cols, optimizer, loss):
  # This is a Sequential model. Graph models can be used in order to create more complex networks. 
  # Teaching Points:
  # 1. Here we utilize the adam optimization algorithm. In order to use the SGD algorithm one could replace the {adam=keras.optimizers.Adadelta(lr=0)} line with  {sgd = SGD(lr=0.0, momentum=0.9, decay=0.0, nesterov=False)} make sure you import the correct optimizer from keras. 
  # 2. This is a binary classification problem so make sure that the correct activation loss function combination is used. For such a problem the sigmoid activation function with the binary cross entropy loss is a good option
  # 3. Since this is a binary problem use   model.add(Dense(1)) NOT 2...
  # 4. For multi class model this code can be easily modified by selecting the softmax as activation function and the categorical cross entropy as loss 

  model = Sequential()
  model.add(Convolution2D(16, 3, 3, padding='same',input_shape=(img_rows, img_cols, 1)))
  model.add(Activation('relu'))
  model.add(Convolution2D(16, 5, 5, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
  model.add(Convolution2D(32, 3, 3, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Convolution2D(64, 5, 5, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Convolution2D(64, 3, 3, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
  model.add(Convolution2D(128, 3, 3, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
  model.add(Flatten())
  model.add(Dense(128, kernel_initializer='he_normal'))
  model.add(Activation('relu'))
  model.add(Dropout(0.5)) 
  model.add(Dense(32, kernel_initializer='he_normal'))
  model.add(Activation('relu'))
  model.add(Dropout(0.5)) 
  model.add(Dense(1))

  model.add(Activation('sigmoid'))

  # learning schedule callback
  
  # Original code had the variable named "adam", but the selected optimizer was adadelta (they are similar optimizers but different slightly)
  model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
  
  print(model.summary())
  return model

def shuffle(X, y):
  perm = np.random.permutation(len(y))
  X = X[perm]
  y = y[perm]
  print("shuffle() new shape for x: ", np.shape(X))
  return X, y

def read_data(image):
  "opens image and converts it to a m*n matrix" 
  image = Image.open(image)
  image = image.getdata()

  image = np.array(image)
  return image.reshape(-1)

def createTrainTestValset(image_dir1, image_dir2):
  Class1_images = file_open(image_dir1,"*.jpg")
  Class2_images = file_open(image_dir2,"*.jpg")

  # Read all the files, and create numpy arrays. 
  Class1_set = np.array([read_data(image) for image in Class1_images])
  Class2_set = np.array([read_data(image) for image in Class2_images])
  X = np.vstack((Class1_set, Class2_set))
  
  X = X / 255.0

  yclass1 = np.zeros((np.shape(Class1_set)[0]))
  yclass2 = np.ones((np.shape(Class2_set)[0]))
  
  y = np.concatenate((yclass1, yclass2))
  
  X,y = shuffle(X, y)

  print("X shape:", np.shape(X)) 
  print("X max:", np.max(X))
  print("Y shape:", np.shape(y)) 
  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
  return X_train, y_train, X_val, y_val 

# Read the images; and split them in three different sets. 
def trainandpredict(optimizer, loss, lrate, checkpoint_file_name='Final', batch_size=64, nb_epoch=5):
  """
  Train the model using some of the inputs, predict the remainder of the inputs using the fitted model and print the report.

  :param optimizer: a keras.optimizers object that the model will recieve during compilation (could also be a string)
  :param loss: a keras.losses object that the model will recieve during compilation (could also be a string)
  :param lrate: a LearningRateScheduler that the model will consider during fitting
  :param checkpoint_file_name: name of the file to save weights to. These values are used and altered during fitting, so 
    be sure to use the correct file for the correct model. Trying to use one weights file for a different model will most likely result
    in an error. Have different weights files for different versions of the model.
  :param batch_size: batch size used during fitting
  :param nb_epoch: number of epochs run during fitting
  """
  img_rows = 32
  img_cols = 32
  CurrentDir = os.getcwd()
  image_dir1 = os.path.abspath(os.path.join(os.path.abspath(os.path.join(CurrentDir, os.pardir)), "Data", "negative_images"))
  image_dir2 = os.path.abspath(os.path.join(os.path.abspath(os.path.join(CurrentDir, os.pardir)), "Data", "positive_images"))

  modeleval = createmodel(img_rows, img_cols, optimizer, loss)

  X_train,y_train, X_val, y_val = createTrainTestValset(image_dir1, image_dir2)

  X_train = X_train.reshape(
    -1,  # number of samples, -1 makes it so that this number is determined automatically
    img_rows,  # first image dimension (vertical)
    img_cols,  # second image dimension (horizontal)
    1,   # 1 color channel, since images are only black and white
  )
  X_val = X_val.reshape(
    -1,  # number of samples, -1 makes it so that this number is determined automatically
    img_rows,  # first image dimension (vertical)
    img_cols,  # second image dimension (horizontal)
    1,   # 1 color channel, since images are only black and white
  )

  filepath = checkpoint_file_name + '.weights.h5'

  # Callbacks
  best_model = ModelCheckpoint(filepath, verbose=1, monitor='val_loss',save_best_only=True,save_weights_only=True)

  try:
    modeleval.load_weights(filepath)
  except FileNotFoundError:
    print(f"Could not find file: {filepath}, assuming this is the first time with this model and will create a new file")
  except ValueError as e:
    print(e)
    print("!!!!!!!ValueError detected, assuming this is a new model and a filepath for a different model's weights was inputted, consider a new weights file")
    sys.exit()

  start = time.time()

  modeleval.fit(X_train, y_train,batch_size=batch_size,epochs=nb_epoch,validation_split=0.1,callbacks=[best_model,lrate],shuffle=True)

  print("Total time to fit:", time.time() - start)

  # Some evaluation Just the basic stuff... 
  #print ("Dir:", dir(modeleval))
  Y_cv_pred = modeleval.predict(X_val, batch_size = 32)
  roc = roc_auc_score(y_val, Y_cv_pred)
  print("ROC:", roc)
  print ("Y_cv_pred:", Y_cv_pred)

  Y_cv_pred[Y_cv_pred>=.5]=1
  Y_cv_pred[Y_cv_pred<.5]=0
   
  target_names = ['class 0', 'class 1']
  # Default notebook output size might not show all information from the result, make sure to expand it or change a setting when viewing
  print("--------------------------")
  print(classification_report(y_val, Y_cv_pred, target_names=target_names,digits=4))





## Run Program 

In [6]:
if __name__ == '__main__':
    # Some of the optimizers provided by keras can take in variations of the
    # keras.optimizers.schedules.LearningRateSchedule objects like
    # keras.optimizers.schedules.ExponentialDecay for example, something to consider.
    # This is NOT the same type of object as the lrate variable (keras.callbacks.LearningRateScheduler),
    # which might be the source of the UserWarning when running, unsure of how to fix this.
    trainandpredict(optimizer=keras.optimizers.Adadelta(learning_rate=0.0), 
                    loss=keras.losses.BinaryCrossentropy,
                    lrate=LearningRateScheduler(step_decay))

c:\Data\VSCodeProjects\MachineLearning\ProjectCode\virtualenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 11, 11, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 11, 11, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 3, 3, 16)       │         6,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 3, 3, 16)       │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_10 (Activation)      │ (None, 3, 3, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 2, 2, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 1, 1, 32)       │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 1, 1, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 1, 1, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 1, 1, 64)       │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 1, 1, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_12 (Activation)      │ (None, 1, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 1, 1, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 1, 1, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_13 (Activation)      │ (None, 1, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 1, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 1, 1, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 1, 1, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_14 (Activation)      │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 195,153 (762.32 KB)

 Trainable params: 194,545 (759.94 KB)

 Non-trainable params: 608 (2.38 KB)

None
shuffle() new shape for x:  (8710, 1024)
X shape: (8710, 1024)
X max: 1.0
Y shape: (8710,)
A total of 15 objects could not be loaded. Example error message for object <Sequential name=sequential_1, built=True>:

"Unable to synchronously open object (object 'vars' doesn't exist)"

List of objects that could not be loaded:
[<Sequential name=sequential_1, built=True>, <Conv2D name=conv2d_6, built=True>, <Conv2D name=conv2d_7, built=True>, <BatchNormalization name=batch_normalization_5, built=True>, <Conv2D name=conv2d_8, built=True>, <BatchNormalization name=batch_normalization_6, built=True>, <Conv2D name=conv2d_9, built=True>, <BatchNormalization name=batch_normalization_7, built=True>, <Conv2D name=conv2d_10, built=True>, <BatchNormalization name=batch_normalization_8, built=True>, <Conv2D name=conv2d_11, built=True>, <BatchNormalization name=batch_normalization_9, built=True>, <Dense name=dense_3, built=True>, <Dense name=dense_4, built=True>, <Dense name=dense_5, built=True>]
!!

c:\Data\VSCodeProjects\MachineLearning\ProjectCode\virtualenv\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adadelta', because it has 2 variables whereas the saved optimizer has 0 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


AttributeError: 'tuple' object has no attribute 'tb_frame'